<a href="https://colab.research.google.com/github/sssteeefaaan/VIII-Semestar/blob/main/Paralelni%20Sistemi/Laboratorijske%20Ve%C5%BEbe/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Instalacija
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!git clone https://github.com/andreinechaev/nvcc4jupyter.git

!pip install ./nvcc4jupyter/

%load_ext nvcc_plugin

In [ ]:
#@title Zadatak 4
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void sub_matrices_GPU(int* A, int* B, int* C);
__global__ void subtract_matrices(int* A, int* B, int* C);
__host__ bool check_result(int* A, int* B, int* GPU_C);
__device__ __host__ int subtract(int a, int b) { return a - b; }

int main(int argc, char** argv)
{
    int* A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    GPU_C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    sub_matrices_GPU(A, B, GPU_C);
 
    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", GPU_C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, GPU_C))
      printf("Matrice su identicne!\n");
    else
      printf("Matrice nisu identicne!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU_C[i * N + j] == subtract(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void sub_matrices_GPU(int *A, int *B, int *C)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    subtract_matrices<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void subtract_matrices(int *A, int* B, int *C)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x,
        thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = thread_id_x; i < N; i+= blockDim.x)
      for(int j = thread_id_y; j < N; j += blockDim.y)
        C[i * N + j] = subtract(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Zadatak 7
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void saberi_matrice_na_GPU(int *A, int* B, int* C, int k, int p);
__global__ void saberi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p);
__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p);
__device__ __host__ int saberi_zameni(int a, int b, __const__ int k, __const__ int p);

int main(int argc, char** argv)
{
    int* A, *B, *C, k = 17190, p = 2904;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    saberi_matrice_na_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == saberi_zameni(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void saberi_matrice_na_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    saberi_matrice<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void saberi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x,
        thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = thread_id_x; i < N; i+= blockDim.x)
      for(int j = thread_id_y; j < N; j += blockDim.y)
        C[i * N + j] = saberi_zameni(A[i * N + j], B[i * N + j], k, p);
}

__device__ __host__ int saberi_zameni(int a, int b, __const__ int k, __const__ int p)
{
    int rez = a + b;
    if(rez > k)
      return p;
    return rez;
}

In [ ]:
#@title Zadatak 8
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void oduzmi_matrice_na_GPU(int *A, int* B, int* C, int k, int p);
__global__ void oduzmi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p);
__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p);
__device__ __host__ int oduzmi_zameni(int a, int b, __const__ int k, __const__ int p);

int main(int argc, char** argv)
{
    int* A, *B, *C, k = 20, p = 666;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    oduzmi_matrice_na_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int *B, int *Za_proveru, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == oduzmi_zameni(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void oduzmi_matrice_na_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    oduzmi_matrice<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void oduzmi_matrice(int *A, int* B, int *C, __const__ int k, __const__ int p)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x,
        thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = thread_id_x; i < N; i+= blockDim.x)
      for(int j = thread_id_y; j < N; j += blockDim.y)
        C[i * N + j] = oduzmi_zameni(A[i * N + j], B[i * N + j], k, p);
}

__device__ __host__ int oduzmi_zameni(int a, int b, __const__ int k, __const__ int p)
{
    int rez = a - b;
    if(rez < k)
      return p;
    return rez;
}

In [ ]:
#@title Zadatak 9
%%cu
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32
#define GRID_SIZE 8

__host__ void skaliraj_na_GPU(int *A, int x, int *C);
__global__ void sklairaj_matricu(int *A, __const__ int x, int * C);
__host__ bool proveri_rezultat(int *A, int x, int *Za_proveru);
__device__ __host__ int skaliraj(int a, int b) { return a * b; }

int main(int argc, char** argv)
{
    int* A, *C, x = 5;
    A = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++)
      for(int j = 0; j < N; j++)
        A[i * N + j] = j * N + i;
 
    skaliraj_na_GPU(A, x, C);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(proveri_rezultat(A, x, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(A);

    return 0;
}

__host__ bool proveri_rezultat(int *A, int x, int *Za_proveru)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (Za_proveru[i * N + j] == skaliraj(A[i * N + j], x));
    return rez;
}

__host__ void skaliraj_na_GPU(int *A, __const__ int x, int *C)
{
    int* dev_A, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    sklairaj_matricu<<<gridSize, blockSize>>>(dev_A, x, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_C);
}

__global__ void sklairaj_matricu(int *A, __const__ int x, int* C)
{
    int thread_id_x = blockIdx.x * blockDim.x + threadIdx.x,
        thread_id_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = thread_id_x; i < N; i+= blockDim.x)
      for(int j = thread_id_y; j < N; j += blockDim.y)
        C[i * N + j] = skaliraj(A[i * N + j], x);
}